In [1]:
import json
from dataclasses import dataclass
from typing import Any, Dict, Literal, Optional, Sequence

import fire
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorForLanguageModeling

from llamafactory.data import MultiModalDataCollatorForSeq2Seq, get_dataset, get_dataset_id, get_template_and_fix_tokenizer,SFTDataCollatorWith4DAttentionMask
from llamafactory.extras.constants import IGNORE_INDEX
from llamafactory.hparams import get_train_args
from llamafactory.model import load_model, load_tokenizer

In [2]:
model_args, data_args, training_args, finetuning_args, _ = get_train_args(
    dict(
        stage='sft',
        model_name_or_path='/data/home/wangys/model/Qwen2.5-0.5B-Instruct',
        adapter_name_or_path = '../LLAMA-backup/LLaMA-Factory/saves/qwen-0.5B/Abt-Buy-Match-P1-short-qwen/',
        dataset='Abt-Buy-Match-P1-short',
        dataset_dir='data',
        template='qwen',
        cutoff_len=400,
        max_samples=None,
        train_on_prompt=False,
        output_dir="output",
        overwrite_cache=False,
        do_train=True,
        # quantization_bit=8
        neat_packing = True,
        enable_liger_kernel = True,
        preprocessing_num_workers=8
    )
)

[WARNING|2024-12-21 18:00:10] llamafactory.hparams.parser:162 >> `neat_packing` requires `packing` is True. Change `packing` to True.
[WARNING|2024-12-21 18:00:10] llamafactory.hparams.parser:162 >> We recommend enable mixed precision training.
[INFO|2024-12-21 18:00:10] llamafactory.hparams.parser:355 >> Process rank: 0, device: cuda:0, n_gpu: 8, distributed training: False, compute dtype: None


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [ ]:
model_args, data_args, training_args, finetuning_args, _ = get_train_args(
    dict(
        stage='sft',
        model_name_or_path='/data/home/wangys/model/Mistral-7B-Instruct-v0.2/',
        adapter_name_or_path = '../LLAMA-backup/LLaMA-Factory/saves/qwen-0.5B/Abt-Buy-Match-P1-short-qwen/',
        dataset='Abt-Buy-Match-P1-short',
        dataset_dir='data',
        template='mistral',
        cutoff_len=400,
        max_samples=None,
        train_on_prompt=False,
        output_dir="output",
        overwrite_cache=False,
        do_train=True,
        # quantization_bit=8
        use_unsloth = False,
        enable_liger_kernel = True,
        preprocessing_num_workers = 1,
        neat_packing=True
    )
)

In [3]:
stage = 'sft'

tokenizer_module = load_tokenizer(model_args)
tokenizer = tokenizer_module["tokenizer"]
template = get_template_and_fix_tokenizer(tokenizer, data_args)
trainset = get_dataset_id(template, model_args, data_args, training_args, stage, **tokenizer_module)


[INFO|configuration_utils.py:677] 2024-12-21 18:00:15,661 >> loading configuration file /data/home/wangys/model/Qwen2.5-0.5B-Instruct/config.json
[INFO|configuration_utils.py:746] 2024-12-21 18:00:15,666 >> Model config Qwen2Config {
  "_name_or_path": "/data/home/wangys/model/Qwen2.5-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.1",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenizatio

[INFO|2024-12-21 18:00:16] llamafactory.data.template:157 >> Add <|im_end|> to stop words.
[INFO|2024-12-21 18:00:16] llamafactory.data.loader:157 >> Loading dataset /data/home/wangys/transfer-er/Pipeline/Abt-Buy/LLM_file/Abt-Buy-Train-Match-P1-short.json...
training example:
input_ids:
[151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 4710, 60256, 3425, 10390, 220, 16, 323, 10390, 220, 17, 525, 2432, 476, 35301, 11, 323, 5157, 2878, 279, 2661, 14566, 382, 5598, 304, 4718, 3561, 382, 3798, 25, 508, 6347, 13086, 24976, 2533, 5097, 3561, 3110, 25, 4913, 5097, 788, 1591, 630, 3030, 220, 16, 2974, 13608, 606, 1210, 364, 94333, 1032, 13891, 480, 33612, 13795, 29067, 12, 24, 16, 20, 516, 364, 4684, 1210, 364, 693, 13891, 480, 11602, 3769, 369, 38116, 7420, 36402, 54768, 22, 18, 15, 33414, 11, 17258, 10842, 220, 16, 23, 15, 11, 17258, 10842, 220, 18, 21, 15, 33414, 11, 17258, 10842, 220, 16, 22, 15, 3424, 11, 17258, 10842, 220, 16, 21, 15, 3424, 11, 17258, 1

In [13]:
trainset = trainset['train_dataset']
data_collator = SFTDataCollatorWith4DAttentionMask(
            template=template, tokenizer=tokenizer, label_pad_token_id=IGNORE_INDEX
        )
dataloader = DataLoader(trainset, batch_size=8, shuffle=False, collate_fn=data_collator, pin_memory=True)
criterion = torch.nn.CrossEntropyLoss(reduction="none")
total_ppl = 0
perplexities = []
batch: Dict[str, "torch.Tensor"]

In [ ]:
torch.save(trainset,'trainset_ER_400.pkl')

In [14]:
model = load_model(tokenizer, model_args, finetuning_args, is_trainable=True)

[INFO|configuration_utils.py:677] 2024-12-21 18:05:10,637 >> loading configuration file /data/home/wangys/model/Qwen2.5-0.5B-Instruct/config.json
[INFO|configuration_utils.py:746] 2024-12-21 18:05:10,641 >> Model config Qwen2Config {
  "_name_or_path": "/data/home/wangys/model/Qwen2.5-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.1",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



[INFO|2024-12-21 18:05:10] llamafactory.model.model_utils.packing:157 >> Using block diagonal attention for sequence packing without cross-attention.
Applied Liger kernels to Qwen2
[INFO|2024-12-21 18:05:10] llamafactory.model.model_utils.liger_kernel:157 >> Liger kernel has been applied to the model.


[INFO|modeling_utils.py:3934] 2024-12-21 18:05:10,777 >> loading weights file /data/home/wangys/model/Qwen2.5-0.5B-Instruct/model.safetensors
[INFO|modeling_utils.py:1670] 2024-12-21 18:05:10,787 >> Instantiating Qwen2ForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1096] 2024-12-21 18:05:10,790 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

[INFO|modeling_utils.py:4800] 2024-12-21 18:05:11,376 >> All model checkpoint weights were used when initializing Qwen2ForCausalLM.

[INFO|modeling_utils.py:4808] 2024-12-21 18:05:11,377 >> All the weights of Qwen2ForCausalLM were initialized from the model checkpoint at /data/home/wangys/model/Qwen2.5-0.5B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen2ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1049] 2024-12-21 18:05:11,380 >> loading configuration file /data/home

[INFO|2024-12-21 18:05:11] llamafactory.model.model_utils.checkpointing:157 >> Gradient checkpointing enabled.
[INFO|2024-12-21 18:05:11] llamafactory.model.model_utils.attention:157 >> Using torch SDPA for faster training and inference.
[INFO|2024-12-21 18:05:11] llamafactory.model.adapter:157 >> Upcasting trainable params to float32.
[INFO|2024-12-21 18:05:11] llamafactory.model.adapter:157 >> Fine-tuning method: LoRA
[INFO|2024-12-21 18:05:11] llamafactory.model.adapter:157 >> Loaded adapter(s): ../LLAMA-backup/LLaMA-Factory/saves/qwen-0.5B/Abt-Buy-Match-P1-short-qwen/
[INFO|2024-12-21 18:05:11] llamafactory.model.loader:157 >> trainable params: 688,128 || all params: 494,720,896 || trainable%: 0.1391


In [16]:
tr_grad_dict = {}
model.eval()
for step,batch in enumerate(tqdm(dataloader)):
    model.zero_grad()
    print(batch['ids'])
    batch = batch.to(model.device)
    outputs = model(**batch)
    loss = outputs.loss

    loss.backward()
    grad_dict={}
    for k, v in model.named_parameters():
        if 'lora_A' in k:
            grad_dict[k]=v.grad.cpu()
        elif 'lora_B' in k:
            # first index of shape indicates low-rank
            grad_dict[k]=v.grad.cpu().T
        else:
            pass
    tr_grad_dict[step]=grad_dict
    # grad_dict

  0%|          | 0/307 [00:00<?, ?it/s]

  0%|          | 1/307 [00:00<00:45,  6.67it/s]

tensor([0, 1, 2, 3, 4, 5, 6, 7])
tensor([ 8,  9, 10, 11, 12, 13, 14, 15])


  1%|          | 3/307 [00:00<00:35,  8.65it/s]

tensor([16, 17, 18, 19, 20, 21, 22, 23])
tensor([24, 25, 26, 27, 28, 29, 30, 31])
tensor([32, 33, 34, 35, 36, 37, 38, 39])


...

tensor([376, 377, 378, 379, 380, 381, 382, 383])
tensor([384, 385, 386, 387, 388, 389, 390, 391])
tensor([392, 393, 394, 395, 396, 397, 398, 399])


 17%|█▋        | 51/307 [00:04<00:23, 10.68it/s]

tensor([400, 401, 402, 403, 404, 405, 406, 407])
tensor([408, 409, 410, 411, 412, 413, 414, 415])
tensor([416, 417, 418, 419, 420, 421, 422, 423])


 17%|█▋        | 52/307 [00:05<00:24, 10.32it/s]


KeyboardInterrupt: 

In [ ]:
python zip.py --data_path ER/semi-text-c/train.csv --save_path zip_selected_data.json --budget 1000 --k1 4500 --k2 200 --k3 100 --n_jobs 64

In [2]:
import pandas as pd
train = pd.read_json('/data/home/wangys/DataSelection-IF/ER/semi-text-c/train.json')
train.to_csv('ER/semi-text-c/train.csv',index=False)

In [8]:
train = pd.read_csv('/data/home/wangys/DataSelection-IF/ER/semi-text-c/train.csv').iloc[:500]

In [9]:
train.to_csv('ER/semi-text-c/train.csv',index=False)

In [ ]:
data

In [ ]:
tr_grad_dict[0]['base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight'].shape

In [ ]:
stage = 'sft'
import torch
trainset = torch.load('trainset_ER_mistral_512.pkl')
tokenizer_module = load_tokenizer(model_args)
tokenizer = tokenizer_module["tokenizer"]
template = get_template_and_fix_tokenizer(tokenizer, data_args)
data_collator = SFTDataCollatorWith4DAttentionMask(
            template=template, tokenizer=tokenizer, label_pad_token_id=IGNORE_INDEX
        )
dataloader = DataLoader(trainset, batch_size=8, shuffle=False, collate_fn=data_collator, pin_memory=True)
criterion = torch.nn.CrossEntropyLoss(reduction="none")
total_ppl = 0
perplexities = []
batch: Dict[str, "torch.Tensor"]

In [ ]:
for step,batch in enumerate(dataloader):
    example = batch
    break

In [ ]:
tr_grad_dict = {}
model.eval()
for step,batch in enumerate(tqdm(dataloader)):
    model.zero_grad()
    # for key in batch.keys():
    #     batch[key] = batch[key][:,-256:]
    batch = batch.to(model.device)
    outputs = model(**batch)

    # loss = sentence_logps.mean()  # 计算平均loss
    loss = outputs.loss
    print(loss.item())
    
    shift_logits: "torch.Tensor" = outputs["logits"][..., :-1, :]
    shift_labels: "torch.Tensor" = batch["labels"][..., 1:]
    loss_mask = shift_labels != IGNORE_INDEX
    flatten_logits = shift_logits.contiguous().view(shift_labels.size(0) * shift_labels.size(1), -1)
    flatten_labels = shift_labels.contiguous().view(-1)
    token_logps: "torch.Tensor" = criterion(flatten_logits, flatten_labels)
    token_logps = token_logps.contiguous().view(shift_logits.size(0), -1)
    sentence_logps = (token_logps * loss_mask).sum(-1) / loss_mask.sum(-1)
    print(sentence_logps.mean())
    loss.backward(retain_graph=True)
    grad_dict={}
    for k, v in model.named_parameters():
        if 'lora_A' in k:
            grad_dict[k]=v.grad.cpu()
        elif 'lora_B' in k:
            # first index of shape indicates low-rank
            grad_dict[k]=v.grad.cpu().T
        else:
            pass
    tr_grad_dict[step]=grad_dict
    # del grad_dict
    # break



In [ ]:
import torch
torch.save(trainset,'trainset_ER.pkl')

In [ ]:
data_collator = MultiModalDataCollatorForSeq2Seq(
            template=template, tokenizer=tokenizer, label_pad_token_id=IGNORE_INDEX
        )
dataloader = DataLoader(trainset, batch_size=8, shuffle=False, collate_fn=data_collator, pin_memory=True)
criterion = torch.nn.CrossEntropyLoss(reduction="none")
total_ppl = 0
perplexities = []
batch: Dict[str, "torch.Tensor"]

In [ ]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

In [ ]:
tr_grad_dict = {}
model.eval()
for step,batch in enumerate(tqdm(dataloader)):
    model.zero_grad()
    # for key in batch.keys():
    #     batch[key] = batch[key][:,-256:]
    batch = batch.to(model.device)
    outputs = model(**batch)

    # loss = sentence_logps.mean()  # 计算平均loss
    loss = outputs.loss
    print(loss.item())
    
    shift_logits: "torch.Tensor" = outputs["logits"][..., :-1, :]
    shift_labels: "torch.Tensor" = batch["labels"][..., 1:]
    loss_mask = shift_labels != IGNORE_INDEX
    flatten_logits = shift_logits.contiguous().view(shift_labels.size(0) * shift_labels.size(1), -1)
    flatten_labels = shift_labels.contiguous().view(-1)
    token_logps: "torch.Tensor" = criterion(flatten_logits, flatten_labels)
    token_logps = token_logps.contiguous().view(shift_logits.size(0), -1)
    sentence_logps = (token_logps * loss_mask).sum(-1) / loss_mask.sum(-1)
    print(sentence_logps.mean())
    loss.backward(retain_graph=True)
    grad_dict={}
    for k, v in model.named_parameters():
        if 'lora_A' in k:
            grad_dict[k]=v.grad.cpu()
        elif 'lora_B' in k:
            # first index of shape indicates low-rank
            grad_dict[k]=v.grad.cpu().T
        else:
            pass
    tr_grad_dict[step]=grad_dict
    # del grad_dict
    # break

In [ ]:
grad_dict['base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight'].shape

In [ ]:
# tr_grad_dict = {}
# model.eval()
# for batch in tqdm(dataloader):
#     model.zero_grad()
#     batch = batch.to(model.device)
#     outputs = model(**batch)
#     shift_logits: "torch.Tensor" = outputs["logits"][..., :-1, :]
#     shift_labels: "torch.Tensor" = batch["labels"][..., 1:]
#     loss_mask = shift_labels != IGNORE_INDEX
#     flatten_logits = shift_logits.contiguous().view(shift_labels.size(0) * shift_labels.size(1), -1)
#     flatten_labels = shift_labels.contiguous().view(-1)
#     token_logps: "torch.Tensor" = criterion(flatten_logits, flatten_labels)
#     token_logps = token_logps.contiguous().view(shift_logits.size(0), -1)
#     sentence_logps = (token_logps * loss_mask).sum(-1) / loss_mask.sum(-1)
#     loss = sentence_logps.mean()  # 计算平均loss
#     loss.backward()
#     grad_dict={}
#     for k, v in model.named_parameters():
#         if 'lora_A' in k:
#             grad_dict[k]=v.grad.cpu()
#         elif 'lora_B' in k:
#             # first index of shape indicates low-rank
#             grad_dict[k]=v.grad.cpu().T
#         else:
#             pass
#     # tr_grad_dict[step]=grad_dict
#     del grad_dict

## qwen-0.5

In [ ]:
model_args, data_args, training_args, finetuning_args, _ = get_train_args(
    dict(
        stage='sft',
        model_name_or_path='/data/home/wangys/model/Qwen2.5-0.5B-Instruct',
        adapter_name_or_path = 'saves/mistral-7b/Abt-Buy-Match-P1-q_proj-qwen',
        dataset='Abt-Buy-Match-P1',
        dataset_dir='data',
        template='qwen',
        cutoff_len=512,
        max_samples=None,
        train_on_prompt=False,
        output_dir="output",
        overwrite_cache=False,
        do_train=True,
        # quantization_bit=8
        use_unsloth = False,
        enable_liger_kernel = True
    )
)

In [ ]:
stage = 'sft'

tokenizer_module = load_tokenizer(model_args)
tokenizer = tokenizer_module["tokenizer"]
template = get_template_and_fix_tokenizer(tokenizer, data_args)
trainset = get_dataset(template, model_args, data_args, training_args, stage, **tokenizer_module)["train_dataset"]


In [ ]:
import torch
torch.save(trainset,'trainset_ER_mistral_512.pkl')

In [ ]:

model = load_model(tokenizer, model_args, finetuning_args, is_trainable=True)
data_collator = MultiModalDataCollatorForSeq2Seq(
            template=template, tokenizer=tokenizer, label_pad_token_id=IGNORE_INDEX
        )
dataloader = DataLoader(trainset, batch_size=16, shuffle=False, collate_fn=data_collator, pin_memory=True)
criterion = torch.nn.CrossEntropyLoss(reduction="none")
total_ppl = 0
perplexities = []
batch: Dict[str, "torch.Tensor"]

In [ ]:
import torch
torch.save(trainset,'trainset_ER_qwen.pkl')

In [ ]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

In [ ]:
tr_grad_dict = {}
model.eval()
for step,batch in enumerate(tqdm(dataloader)):
    model.zero_grad()
    batch = batch.to(model.device)
    outputs = model(**batch)
    loss = outputs.loss
    # shift_logits: "torch.Tensor" = outputs["logits"][..., :-1, :]
    # shift_labels: "torch.Tensor" = batch["labels"][..., 1:]
    # loss_mask = shift_labels != IGNORE_INDEX
    # flatten_logits = shift_logits.contiguous().view(shift_labels.size(0) * shift_labels.size(1), -1)
    # flatten_labels = shift_labels.contiguous().view(-1)
    # token_logps: "torch.Tensor" = criterion(flatten_logits, flatten_labels)
    # token_logps = token_logps.contiguous().view(shift_logits.size(0), -1)
    # sentence_logps = (token_logps * loss_mask).sum(-1) / loss_mask.sum(-1)
    # loss = sentence_logps.mean()  # 计算平均loss
    loss.backward()
    grad_dict={}
    for k, v in model.named_parameters():
        if 'lora_A' in k:
            grad_dict[k]=v.grad.cpu()
        elif 'lora_B' in k:
            # first index of shape indicates low-rank
            grad_dict[k]=v.grad.cpu().T
        else:
            pass
    tr_grad_dict[step]=grad_dict
    # grad_dict

In [ ]:
tr_grad_dict[0]['base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight'].shape

In [ ]:
grad_dict['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight'].shape